`pidgin.display` is a collection of display rules for IPython objects.  

`StringFormatter` is a class that adds reules for string `object`s.

In [1]:
    import abc, fnmatch, io, dataclasses, functools, html, IPython, jinja2, inspect, mistune, textwrap, collections, inspect, importlib, types, base64, string, traitlets
    from mimetypes import guess_type; guess = lambda x: guess_type(x)[0]
    with __import__('importnb').Notebook():
        try:  from . import loader
        except: import loader
    with loader.PidginImporter():
        try: 
            from .templating import format_images
            from .weave import PidginDisplay, code_formatting
        except: 
            from weave import PidginDisplay, code_formatting
            from templating import format_images
    ip = IPython.get_ipython()
    
    if __name__ == '__main__':
        %reload_ext pidgin


In [2]:
    def graphviz(ip, str): 
        try: import graphviz
        except: return
        if '->' in str and not str.startswith('di'): str = 'di' + str
        return graphviz.Source(str)

In [3]:
    def eval_shorthand_ipython(ip, str):
        return eval(ip.input_transformer_manager.transform_cell(str), ip.user_ns, ip.user_global_ns)

In [4]:
    def embed(ip, str):
        type = guess(str) or ''
        if str.startswith('http'): 
            if type.startswith('image') and not type.endswith('svg'):
                return IPython.display.Image(str)
            return IPython.display.IFrame(str, '100%', '500px')


In [5]:
    class StringFormatterBase(abc.ABCMeta):
        level = 0
        def __enter__(cls): cls.level += 1
        def __exit__(cls, *exc): cls.level -= 1

        def format(cls, object, include=None, exclude=None, display=False):
            if object in ip.user_ns: 
                value = ip.user_ns.get(object)
                if value != object: return ip.display_formatter.format(value)
            with cls:
                for type, callable in cls.formatters.items():
                    if fnmatch.fnmatch(object, type):
                        result = callable(ip, object)
                        if result: return ip.display_formatter.format(result)
                if cls.level == 1: 
                    return ip.display_formatter.format(PidginDisplay(object, ip=ip))

In [6]:
    class StringFormatter(metaclass=StringFormatterBase):
        formatters = {'https://*' : embed,
                      'http://*' : embed,
                      'graph {*}': graphviz, 
                      'digraph {*}': graphviz,
                      ';[! ]*': eval_shorthand_ipython,
                      ',[! ]*': eval_shorthand_ipython,
                     }

In [7]:
    def format_object(object):
        if not hasattr(object, '__name__'): return

        ip = IPython.get_ipython()

        for _ in dir(object):
            if _.startswith('_repr') or _.startswith('_ipython'):
                return 

        if isinstance(object, type) and not hasattr(object, '__file__') and object.__doc__ and len(object.__mro__) == 2:
            return ip.display_formatter.format(object.__doc__)[0]
        
        file = ""
        string_form = str(object)
        try: title = ip.inspector._get_info(object).get('text/plain')
        except: title = ""
        
        
        if IPython.get_ipython().weave.safe:
            file = object.__file__ if hasattr(object, '__file__') else ''

        return {
            'text/html': F"""<a href="{"file:///"+file}" title="{
                    code_formatting.format_title(title)
            }"><code>{mistune.escape(string_form)}</code></a>""", 'text/plain': string_form}

In [8]:
    def format_bytes(bytes, object):
        return {
            'text/html': format_images('image/png', {'image/png': bytes}),
            'text/plain': repr(object),}, {}

In [9]:
    def show_axes(object):
        ip = IPython.get_ipython()
        import matplotlib.backends.backend_agg
        bytes = io.BytesIO()
        matplotlib.backends.backend_agg.FigureCanvasAgg(
            getattr(object, 'figure', object)
        ).print_png(bytes)
        return format_bytes(bytes.getvalue(), object)

In [10]:
    def show_sympy_axes(object): 
        s = io.BytesIO()
        object.save(s)
        return format_bytes(s.getvalue(), object)

In [11]:
    def format_frame(DataFrame):
        return {
            'text/html': F"<span>{DataFrame._repr_html_().lstrip('<div>').rstrip('</div>')}</span>",
            'text/plain': repr(DataFrame)
        }, {}

In [12]:
    formatters_by_name = (
        ('matplotlib.figure', 'Axes', show_axes), 
        ('matplotlib.axes._subplots', 'AxesSubplot', show_axes),
        ('pandas.core.frame', 'DataFrame', format_frame),
        ('sympy.plotting.plot', 'Plot', show_sympy_axes),
    )

In [13]:
    def load_ipython_extension(ip): 
        mimebundle_formatter = ip.display_formatter.mimebundle_formatter
        for object in formatters_by_name: mimebundle_formatter.for_type_by_name(*object)  
        mimebundle_formatter.for_type(str, StringFormatter.format)
        mimebundle_formatter.for_type(type, format_object)
        mimebundle_formatter.for_type(types.ModuleType, format_object)


    def unload_ipython_extension(ip=None): 
        for object in (type, types.ModuleType, str): 
            try: ip.display_formatter.mimebundle_formatter.type_printers.pop(object)
            except: ...
        for module, name, callable in formatters_by_name:
            try: ip.display_formatter.mimebundle_formatter.deferred_printers.pop((module, name))
            except: ...

    if __name__ == '__main__':
        %reload_ext tangle
        %reload_ext weave
        unload_ipython_extension(get_ipython())        
        load_ipython_extension(get_ipython())